# FAKEREVIEW PROJECT - MP12

## Purpose
   - Training and testing the different methodology to identify the deceptive review

## Methodology
   - Machine learning algos
   - Deep Learning 


## WIP - improvements
Tried with LightBGM 

Notable TODOs:
- todo 1;
- todo 2;
- todo 3.

## Results
   - Achieved 80.6% in Precision and Recall 

## Suggested next steps
   - Try another preprocessing method 
   - Optimizing the hyperparameter



## Library import
We import all the required Python libraries

In [32]:
# Data manipulation
import pandas as pd
import numpy as np
#import xgboost as xgb
#from catboost import Pool,CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# showing multiple outputs 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pandarallel import pandarallel

# Initialization
pandarallel.initialize(progress_bar=True)

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef,roc_auc_score

#import lazypredict
#from lazypredict.Supervised import LazyClassifier


INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
#!pip3 install pandarallel
#!pip3 list

## Local library import
We import all the required local libraries libraries

In [3]:
# Include local library paths
import sys
# sys.path.append('path/to/local/lib') # uncomment and fill to import local libraries

# Import local libraries

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.


# Data import
We retrieve all the required data for the analysis.

In [4]:
#read the data
deceptive_opinion = pd.read_csv('deceptive-opinion.csv')
Yelp_review_sentiments = pd.read_excel('Yelp Labelled Review Dataset with Sentiments and Features.xlsx',engine='openpyxl')

# Data processing
Put here the core of the notebook. Feel free di further split this section into subsections.

In [5]:
#convert the label to 0:truthful and 1:deceptive
Yelp_review_sentiments['deceptive'] = Yelp_review_sentiments['Spam(1) and Not Spam(0)'].map({1: 'deceptive', 0: 'truthful'})
Yelp_review_sentiments['text'] = Yelp_review_sentiments["Review"]

In [6]:
deceptive_opinion
Yelp_review_sentiments

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...
...,...,...,...,...,...
1595,deceptive,intercontinental,negative,MTurk,Problems started when I booked the InterContin...
1596,deceptive,amalfi,negative,MTurk,The Amalfi Hotel has a beautiful website and i...
1597,deceptive,intercontinental,negative,MTurk,The Intercontinental Chicago Magnificent Mile ...
1598,deceptive,palmer,negative,MTurk,"The Palmer House Hilton, while it looks good i..."


,User_id,Product_id,Rating,Date,Review,Spam(1) and Not Spam(0),Sentiment,Features,deceptive,text
0,923,0,3,2014-01-30,The food at snack is a selection of popular Gr...,1,Positive,"['appetizer tray', 'greek salad', 'main courses']",deceptive,The food at snack is a selection of popular Gr...
1,924,0,3,2011-05-05,This little place in Soho is wonderful. I had ...,1,Positive,"['little place', 'soho', 'lamb sandwich', 'soh...",deceptive,This little place in Soho is wonderful. I had ...
2,925,0,4,2011-12-30,ordered lunch for 15 from Snack last Friday. Ã...,1,Positive,"['snack', 'regular company lunch list']",deceptive,ordered lunch for 15 from Snack last Friday. Ã...
3,926,0,4,2012-10-04,This is a beautiful quaint little restaurant o...,1,Positive,"['beautiful quaint', 'pretty street', 'great p...",deceptive,This is a beautiful quaint little restaurant o...
4,927,0,4,2014-02-06,Snack is great place for a Ã‚Â casual sit down...,1,Positive,"['snack', 'great place', 'Ã¢ casual', 'cold wi...",deceptive,Snack is great place for a Ã‚Â casual sit down...
...,...,...,...,...,...,...,...,...,...,...
355205,161146,349,1,2012-10-04,The aircondition makes so much noise and its ...,0,Negative,[],truthful,The aircondition makes so much noise and its ...
355206,116424,349,1,2013-05-27,Even though the pictures show very clean room...,0,Negative,"['clean rooms', 'actual room', 'o clock']",truthful,Even though the pictures show very clean room...
355207,161147,349,2,2011-03-03,Backyard of the hotel is total mess shouldn t...,0,Negative,"['backyard', 'total mess shouldn t']",truthful,Backyard of the hotel is total mess shouldn t...
355208,97930,349,2,2014-07-29,You When I booked with your company on line y...,0,Negative,"['s room', 'villa suite theough', 'wife s 40th...",truthful,You When I booked with your company on line y...


In [7]:
#filter deceptive opinion dataset with just text and label
sub_deceptive_opinion = deceptive_opinion[["deceptive", "text"]]

In [8]:
#filter Yelp dataset with just text and label
sub_Yelp_review_sentiments = Yelp_review_sentiments[["deceptive", "text"]]

In [9]:
#create the subset for each label 
sub_Yelp_deceptive = sub_Yelp_review_sentiments[sub_Yelp_review_sentiments["deceptive"] == "deceptive"].reset_index(drop = True)
sub_Yelp_truthful = sub_Yelp_review_sentiments[sub_Yelp_review_sentiments["deceptive"] == "truthful"].reset_index(drop = True).iloc[:36133,:]

In [10]:
#sub dataset of Yelp with around 36100 observations - balance class
concat_data = pd.concat([sub_deceptive_opinion, sub_Yelp_deceptive,sub_Yelp_truthful],ignore_index=True)

In [2]:
#concat_data.to_csv('concat_data.csv',index=False)
concat_data = pd.read_csv('concat_data.csv')
concat_data

,deceptive,text
0,1,We stayed for a one night getaway with family ...
1,1,Triple A rate with upgrade to view room was le...
2,1,This comes a little late as I'm finally catchi...
3,1,The Omni Chicago really delivers on all fronts...
4,1,I asked for a high floor away from the elevato...
...,...,...
73861,1,Excellent food and awesome 5 star service from...
73862,1,YUMMY. Although I haven't had ramen from many ...
73863,1,The soup is flavorful and deeply taste behind ...
73864,1,It was a snowy midweek afternoon that we decid...


sub_deceptive_opinion = deceptive_opinion[["deceptive", "text"]]
sub_Yelp_review_sentiments = Yelp_review_sentiments[["deceptive", "text"]]
concat_data = pd.concat([sub_deceptive_opinion, sub_Yelp_review_sentiments],ignore_index=True)

In [3]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import string

def clean_text(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,remove special characters
    and remove words containing numbers.
    '''
    #print(text)
    if type(text) is str:
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub("\\W"," ",text) # remove special chars
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
    else:
        text = str(text)
    
    return text
    

In [4]:
import string
#nltk.download('stopwords')
from nltk.corpus import stopwords

def remove_stopwords(text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure','ã','â']
    # Check characters to see if they are in punctuation
    nopunc = [char for char in text if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [5]:
stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

In [6]:
def preprocess_data(text):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    text = remove_stopwords(text)
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    
    return text

In [7]:
#preprocessing the text
concat_data['cleaned_text']=concat_data['text'].parallel_apply(preprocess_data)


In [8]:
concat_data

,deceptive,text,cleaned_text
0,1,We stayed for a one night getaway with family ...,stay one night getaway famili thursday tripl a...
1,1,Triple A rate with upgrade to view room was le...,tripl rate upgrad view room less also includ b...
2,1,This comes a little late as I'm finally catchi...,come littl late final catch review past sever ...
3,1,The Omni Chicago really delivers on all fronts...,omni chicago realli deliv front spacious room ...
4,1,I asked for a high floor away from the elevato...,ask high floor away elev got room pleasant dec...
...,...,...,...
73861,1,Excellent food and awesome 5 star service from...,excel food awesom star servic moment walk leav...
73862,1,YUMMY. Although I haven't had ramen from many ...,yummi although ramen mani differ place probabl...
73863,1,The soup is flavorful and deeply taste behind ...,soup flavor deepli tast behind love system ser...
73864,1,It was a snowy midweek afternoon that we decid...,snowi midweek afternoon decid drop bowl ramen ...


In [17]:
#concat_data.to_csv('concatdata2.csv',index=False)

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

x=concat_data['cleaned_text']
y=concat_data['deceptive']
le=LabelEncoder()
y=le.fit_transform(y)



In [10]:
#X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=.1,random_state =23,stratify=y)
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=.05,random_state =23)

In [78]:
#from sklearn.feature_extraction.text import CountVectorizer
#from nltk.stem import WordNetLemmatizer


#class LemmaTokenizer(object):
    #def __init__(self):
        #self.wnl=WordNetLemmatizer()
    #def __call__(self,doc):
        #return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    
#cv=CountVectorizer(analyzer='word',input='content',stop_words='english', ngram_range=(1,2),max_features=60000,min_df=3,tokenizer=LemmaTokenizer()) #60000 reached 79.6
#X_train=cv.fit_transform(X_train)
#X_test=cv.transform(X_test)

In [11]:
#Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english', ngram_range=(1,2),max_features=60000) #60000 reached 79.6
X_train=cv.fit_transform(X_train)
X_test=cv.transform(X_test)

In [74]:
#convert the Countvectorize to compressed pickle file 
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + ".pbz2", "w") as f: 
        cPickle.dump(data, f)

In [75]:
#compressed_pickle('cvtransform.pkl', cv) 

In [12]:
Y_train.shape
X_train.shape
X_test.shape

(70172,)

(70172, 60000)

(3694, 60000)

In [13]:
#Transform a count matrix to a normalized tf or tf-idf representation
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train)
tfidf_transformer.transform(X_train)

TfidfTransformer()

<70172x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 3959841 stored elements in Compressed Sparse Row format>

## Train the model

In [15]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.tree import DecisionTreeClassifier

In [33]:
#!pip install lightgbm

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')   
Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')



model = LGBMClassifier(random_state=24,boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.2, n_estimators=300, device = "cpu")

LGBMC = model.fit(X_train, Y_train)

# Do the prediction
y_predict =LGBMC.predict(X_test)
print(confusion_matrix(Y_test,y_predict))
recall=recall_score(Y_test,y_predict,average='macro')
precision=precision_score(Y_test,y_predict,average='macro')
f1score=f1_score(Y_test,y_predict,average='macro')
accuracy=accuracy_score(Y_test,y_predict)
matthews = matthews_corrcoef(Y_test,y_predict) 
roc_auc = roc_auc_score(Y_test,y_predict,average='macro') 
print('Accuracy: '+ str(accuracy))
print('Macro Precision: '+ str(precision))
print('Macro Recall: '+ str(recall))
print('Macro F1 score:'+ str(f1score))
print('MCC:'+ str(matthews))
print('roc_auc:'+ str(roc_auc))

[[1556  287]
 [ 459 1392]]
Accuracy: 0.7980508933405522
Macro Precision: 0.8006366781597813
Macro Recall: 0.798150784738082
Macro F1 score:0.7976520588541965
MCC:0.5987823027257537
roc_auc:0.7981507847380821


### Hyperparameter tuning

In [17]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV

Tips:set n_jobs=-1 in random_cv = RandomizedSearchCV(n_jobs=-1) to maximize all cores

In [24]:
lightBGM_clf = LGBMClassifier(random_state=24,n_estimators=5000, device = "cpu")

param_distribs = [
    { # lightBGM
        'learning_rate': [0.005, 0.1, 0.2],
        'boosting_type': ['gbdt', 'dart', 'goss'],
        'num_leaves': [10, 30,50],
        'subsample': sp_uniform(loc=0.2, scale=0.8),
        'max_depth': [10, 20, 25]
        }]

rnd_search_cv = RandomizedSearchCV(lightBGM_clf, param_distribs, n_iter=10, cv=3,refit=True,
                                   random_state=314,verbose=True,scoring='roc_auc')

rnd_search_cv.fit(X_train, Y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=LGBMClassifier(device='cpu', n_estimators=5000,
                                            random_state=24),
                   param_distributions=[{'boosting_type': ['gbdt', 'dart',
                                                           'goss'],
                                         'learning_rate': [0.005, 0.1, 0.2],
                                         'max_depth': [10, 20, 25],
                                         'num_leaves': [10, 30, 50],
                                         'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3c74c142d0>}],
                   random_state=314, scoring='roc_auc', verbose=True)

In [26]:
print('Best score reached: {} with params: {} '.format(rnd_search_cv.best_score_, rnd_search_cv.best_params_))

Best score reached: 0.8738434568482675 with params: {'boosting_type': 'dart', 'learning_rate': 0.1, 'max_depth': 25, 'num_leaves': 30, 'subsample': 0.2450155891961434} 


Best score reached: 0.8738434568482675 with params: {'boosting_type': 'dart', 'learning_rate': 0.1, 'max_depth': 25, 'num_leaves': 30, 'subsample': 0.2450155891961434} 

In [34]:
rnd_search_cv.best_estimator_

LGBMClassifier(boosting_type='dart', device='cpu', max_depth=25,
               n_estimators=5000, num_leaves=30, random_state=24,
               subsample=0.2450155891961434)

In [35]:
rnd_search_params = {'boosting_type': 'dart', 'learning_rate': 0.1, 'max_depth': 25, 'num_leaves': 30, 'subsample': 0.2450155891961434}
#LGBMmodel = rnd_search_cv.best_estimator_
LGBMmodel = LGBMClassifier(boosting_type='dart', device='cpu', max_depth=25,
               n_estimators=5000, num_leaves=30, random_state=24,
               subsample=0.2450155891961434)
LGBMmodel.fit(X_train, Y_train)

LGBMClassifier(boosting_type='dart', device='cpu', max_depth=25,
               n_estimators=5000, num_leaves=30, random_state=24,
               subsample=0.2450155891961434)

In [38]:
# Do the prediction
y_predict =LGBMmodel.predict(X_test)
y_prob = LGBMmodel.predict_proba(X_test)
print(confusion_matrix(Y_test,y_predict))
recall=recall_score(Y_test,y_predict,average='macro')
precision=precision_score(Y_test,y_predict,average='macro')
f1score=f1_score(Y_test,y_predict,average='macro')
accuracy=accuracy_score(Y_test,y_predict)
matthews = matthews_corrcoef(Y_test,y_predict) 
roc_auc = roc_auc_score(Y_test,y_predict,average='macro') 
print('Accuracy: '+ str(accuracy))
print('Macro Precision: '+ str(precision))
print('Macro Recall: '+ str(recall))
print('Macro F1 score:'+ str(f1score))
print('MCC:'+ str(matthews))
print('roc_auc:'+ str(roc_auc))

[[1564  279]
 [ 442 1409]]
Accuracy: 0.8048186247969681
Macro Precision: 0.8071883283797896
Macro Recall: 0.8049132714993552
Macro F1 score:0.8044743753143442
MCC:0.612097371903196
roc_auc:0.8049132714993553


In [45]:
#showing the probability of the label 
y_prob
y_prob[:,1]*100 #prob of deceptive review

array([[0.75947906, 0.24052094],
       [0.03751174, 0.96248826],
       [0.40323909, 0.59676091],
       ...,
       [0.25773305, 0.74226695],
       [0.82143092, 0.17856908],
       [0.03164694, 0.96835306]])

array([24.05209372, 96.24882644, 59.67609068, ..., 74.22669475,
       17.85690758, 96.8353063 ])

**Note**: The first number represent the prob of label 0

### Export the model

In [43]:
import pickle
# open a file, where you ant to store the data
file = open('lightBGM.pkl', 'wb')

# dump information to that file
pickle.dump(LGBMmodel, file)

# Extra code for References

In [70]:
import spacy
from spacy_langdetect import LanguageDetector
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
text = 'This is an english text có cả tieng việt i hope they could rethink '
doc = nlp(text)
# document level language detection. Think of it like average language of the document!
print(doc._.language)
# sentence level language detection
for sent in doc.sents:
   print(sent, sent._.language)

{'language': 'en', 'score': 0.8571396576334295}
This is an english text có cả tieng việt {'language': 'vi', 'score': 0.8571415221786176}
i hope they could rethink {'language': 'en', 'score': 0.999995906929855}


In [71]:
doc._.language["language"]

'en'

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2